In [19]:
import bs4
import json
import tqdm
import time
import lxml
import requests
from random import shuffle
from get_item_info import get_item_info

sess = requests.Session()
sess.headers.update({'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.59 Safari/537.36'})

product = {
    'url': 'https://www.aliexpress.com/item/33022387453.html?spm=a2g0o.productlist.0.0.630e72dccWUVuJ&algo_pvid=33fe14d5-20aa-4f57-aa23-04c854e5baff&algo_expid=33fe14d5-20aa-4f57-aa23-04c854e5baff-0&btsid=0ab50f4415818504612624832eaffb&ws_ab_test=searchweb0_0,searchweb201602_,searchweb201603_',
    'id': '33022387453',
    'cat': 'Women\'s Clothing & Accessories',
    'subcat': 'Dresses'
}
cookies = None
proxy = None
# get_item_info(product, sess, None, None)

In [20]:
item_url = product['url']
item_id = item_url.split('?')[0].split('/')[-1].split('.html')[0]
item_id

'33022387453'

In [21]:
r = sess.get(item_url+'&isOrigTitle=true',
                 cookies=cookies, proxies=proxy, timeout=5.)

In [24]:
item_soup = bs4.BeautifulSoup(r.text, 'lxml')
title = item_soup.select_one('title').text

In [27]:
colors = []
for prop in item_soup.select('div.sku-property'):
    if prop.select_one('dt.sku-title').text == 'Color:':
        for color_item in prop.select('li'):
            if color_item.select_one('img') is None:
                color_url = ''
            else:
                color_url = color_item.select_one('img').attrs['bigpic']
            colors.append({'color': color_item.select_one('a').attrs['title'],
                           'url': color_url})

    if prop.select_one('dt.sku-title').text == 'Size:':
        sizes = [x.text for x in prop.select('li')]


In [32]:
item_soup.select('div.sku-property')

[]

In [23]:
item_soup

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<title>Marwin New Coming Spring Summer Holiday Dress Cross Spaghetti Strap Open Back Solid Beach Style Ankle Length Women Dresses-in Dresses from Women's Clothing on AliExpress </title>
<meta content="Dresses, Cheap Dresses, Marwin New Coming Spring Summer Holiday Dress Cross Spaghetti Strap Open Back Solid Beach Style Ankle Length Women Dresses" name="keywords"/>
<meta content="Cheap Dresses, Buy Directly from China Suppliers:Marwin New-Coming Spring Summer Holiday Dress Cross Spaghetti Strap Open Back Solid Beach Style Ankle-Length Women Dresses
Enjoy ✓Free Shipping Worldwide! ✓Limited Time Sale ✓Easy Return." name="description"/>
<meta content="8daa66079a8aa29e-f219f934a1051f5a-ge19f8e1eaa3bf94b-e" name="google-translate-customization"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
<meta conten

In [ ]:
def get_item_info(product, sess, proxy, cookies):
    item_url = product['url']
    item_id = item_url.split('?')[0].split('/')[-1].split('.html')[0]

    r = sess.get(item_url+'&isOrigTitle=true',
                 cookies=cookies, proxies=proxy, timeout=5.)
    item_soup = bs4.BeautifulSoup(r.text, 'lxml')
    title = item_soup.select_one('h1.product-name').text
    price = item_soup.select_one('span#j-sku-price').text

    try:
        price_disc = item_soup.select_one('span#j-sku-discount-price').text
    except:
        price_disc = None

    rating = float(item_soup.find(itemprop='ratingValue').text)
    review_num = int(item_soup.find(itemprop='reviewCount').text)
    order_num = int(''.join([x for x in item_soup.select_one(
        'span#j-order-num').text if x.isdigit()]))

    props = {}
    for prop in item_soup.select('li.property-item'):
        props[prop.select_one('span.propery-title').text[:-1]
              ] = prop.select_one('span.propery-des').text

    colors = []
    for prop in item_soup.select('dl.p-property-item'):
        if prop.select_one('dt.p-item-title').text == 'Color:':
            for color_item in prop.select('li'):
                if color_item.select_one('img') is None:
                    color_url = ''
                else:
                    color_url = color_item.select_one('img').attrs['bigpic']
                colors.append({'color': color_item.select_one('a').attrs['title'],
                               'url': color_url})

        if prop.select_one('dt.p-item-title').text == 'Size:':
            sizes = [x.text for x in prop.select('li')]

    images = []
    for image_item in item_soup.select('span.img-thumb-item'):
        images.append(image_item.select_one(
            'img').attrs['src'].replace('_50x50.jpg', '.jpg'))

    store = item_soup.select_one('a.store-lnk').text

    r = sess.get('http:'+item_soup.text.split('window.runParams.descUrl="')
                 [-1].split('"')[0], cookies=cookies, proxies=proxy, timeout=5.)
    desc_soup = bs4.BeautifulSoup(r.text, 'lxml')
    desc_images = [x.attrs['src']
                   for x in desc_soup.select('img') if 'src' in x.attrs]
    feedback_url = 'http:' + \
        item_soup.select_one('div#feedback').select_one(
            'iframe').attrs['thesrc']

    feedbacks = []
    page = 1
    while True:
        r = sess.get(feedback_url+'&page=%d&withPictures=true' %
                     page, cookies=cookies, proxies=proxy, timeout=10.)
        page += 1
        feedback_soup = bs4.BeautifulSoup(r.text, 'lxml')
        if len(feedback_soup.select('div.feedback-item')) == 0:
            break

        for feedback_item in feedback_soup.select('div.feedback-item'):
            user_country = feedback_item.select_one('div.user-country').text
            feedback_rating = int(feedback_item.select_one(
                'span.star-view').select_one('span').attrs['style'].split('width:')[-1].split('%')[0])

            feedback_props_tmp = []
            for x in feedback_item.select_one('div.user-order-info').text.split('\n'):
                if x.strip() != '':
                    feedback_props_tmp.append(x.strip().replace(' ', ''))

            feedback_props = {}
            for i, x in enumerate(feedback_props_tmp):
                if i % 2 == 0:
                    prop_name = x[:-1]
                else:
                    feedback_props[prop_name] = x

            try:
                feedback_assignment = feedback_item.select_one(
                    'span.product-feedback-info').text.strip()
            except:
                feedback_assignment = None

            personal_info = {}
            for personal_item in feedback_item.select('span.buyer-personal-info-item'):
                personal_item_title = personal_item.select_one(
                    'span.buyer-personal-info-title').text[:-1]
                personal_item_value = ''.join(
                    personal_item.findAll(text=True, recursive=False)).strip()
                personal_info[personal_item_title] = personal_item_value

            feedback_images = [x.attrs['data-src']
                               for x in feedback_item.select('li.pic-view-item')]
            feedback_text = feedback_item.select_one(
                'dt.buyer-feedback').text.strip()
            feedback_time = feedback_item.select_one('dd.r-time').text.strip()

            feedbacks.append({
                'country': user_country,
                'rating': feedback_rating,
                'props': feedback_props,
                'assign': feedback_assignment,
                'person': personal_info,
                'imgs': feedback_images,
                'txt': feedback_text,
                'time': feedback_time,
            })

    item_info = {
        'url': item_url,
        'id': item_id,
        'title': title,
        'price': price,
        'price_disc': price_disc,
        'rating': rating,
        'review_num': review_num,
        'order_num': order_num,
        'props': props,
        'colors': colors,
        'images': images,
        'store': store,
        'desc_images': desc_images,
        'feedbacks': feedbacks,
        'cat': product['cat'],
        'subcat': product['subcat'],
    }

    return item_info


In [33]:
!pip install aliexpress-page-parser


     |████████████████████████████████| 2.0 MB 4.4 MB/s eta 0:00:01
  Created wheel for dukpy: filename=dukpy-0.2.2-cp37-cp37m-macosx_10_9_x86_64.whl size=1484281 sha256=2a9a4e756f3c098033f7c09c69108bde58883fcc4b2a63411b4f9463d7032b0d
  Stored in directory: /Users/tianxiangliu/Library/Caches/pip/wheels/d6/21/13/518caab35338c69f6aec62567e7d3c6ea9bccd07bb93f8edb6
Successfully built dukpy
